In [ ]:
PROJECT_ID = "nasa-genie-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [ ]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [ ]:
# import tempfile
# import os
from typing import List, Optional, Tuple, Dict
# from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from services.query.query_service import retrieve_references

In [ ]:
# # do this once per vm environment
# #!python -m spacy download en_core_web_sm

# nlp = spacy.load("en_core_web_sm")
# storage_client = storage.Client(project=PROJECT_ID)

## Setup

In [8]:
from common.models import QueryEngine, QueryDocument, QueryDocumentChunk

In [9]:
# from firebase
user_id = "bfrv1ifGrvAiJ9sWENHE"

# name of existing query engines
qe_name_one = "nasa-search-integrated-v2"
qe_name_one = "nasa-search-gcs"

# qe_name_two = "nasa-genie-dev-source-docs-02"

# qe_id_one = "LiHrECAO7M0pPQ1SDyjS"

# prompt_one = "Where is the Barringer Meteor Crater?"

## Understand QueryResult object model

In [10]:
qe_one = QueryEngine.find_by_name(qe_name_one)
# qe_two = QueryEngine.find_by_name(qe_name_two)

In [11]:
qe_one.id

'6kcXnzQGxUkWHTcusn5j'

In [ ]:
# from services.query.query_service import retrieve_references
# from services.query.query_service import query_search

# query_refs = retrieve_references(
#     prompt_one,
#     qe_one,
#     user_id
# )
# query_refs = query_search(
#     qe_one,
#     prompt_one,
# )

# Exception: Failed to create vector extension: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "postgres"

In [12]:
query_docs = QueryDocument.find_by_query_engine_id(qe_one.id)

In [13]:
for doc in query_docs:
    print(f"[{doc.index_start}, {doc.index_end}] {doc.id} {doc.doc_url}")
    for i in range(doc.index_start, doc.index_end):
        doc_chunk = QueryDocumentChunk.find_by_index(qe_one.id, i)
        print(f"{doc_chunk.index:8} {doc_chunk.query_document_id} {doc_chunk.clean_text[0:50]}")

[19438, 19705] N6HzPqe4lnSmgjrq60a2 /b/nasa-genie-dev-ntrs/o/Orion%2Ficcfd11_predictingOrionLaunchAbortAcoustics_v2.pdf
   19438 N6HzPqe4lnSmgjrq60a2 Eleventh International Conference onComputational 
   19439 N6HzPqe4lnSmgjrq60a2 Eleventh International Conference onComputational 
   19440 N6HzPqe4lnSmgjrq60a2 The primary objectives are to establish bestpracti
   19441 N6HzPqe4lnSmgjrq60a2 Excellent agreement is obtained between CFD predic
   19442 N6HzPqe4lnSmgjrq60a2 A simplied (static and at rest)simulation of a pad
   19443 N6HzPqe4lnSmgjrq60a2 Simulation results for a supersonic ascent abortig
   19444 N6HzPqe4lnSmgjrq60a2 Overall,the CFD results demonstrate that scale-res
   19445 N6HzPqe4lnSmgjrq60a2 Orionwill be propelled into space by the powerful 
   19446 N6HzPqe4lnSmgjrq60a2 The launch abort system(LAS), detailed in Figure 2
   19447 N6HzPqe4lnSmgjrq60a2 The LAS consists of a tower where three rocket mot
   19448 N6HzPqe4lnSmgjrq60a2 Together, the crew module and theLAS for

KeyboardInterrupt: 

In [14]:
def get_chunks(qe_id):
    doc_chunks = []
    query_docs = QueryDocument.find_by_query_engine_id(qe_id)
    for doc in query_docs:
        for i in range(doc.index_start, doc.index_end):
            doc_chunk = QueryDocumentChunk.find_by_index(qe_id, i)
            doc_chunks.append(doc_chunk)
        break
    return doc_chunks

## Reranking

In [15]:
from rerankers import Reranker
from bs4 import BeautifulSoup

In [16]:
chunks = get_chunks(qe_one.id)
print(f"Got {len(chunks)} chunks")

Got 267 chunks


In [17]:
ENCODER_MODELS = {
    "cross-encoder": Reranker('cross-encoder', verbose=0),
    "colbert": Reranker('colbert', verbose=0)
}

Loading default cross-encoder model for language en
If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1
Loading default colbert model for language en
Loading ColBERTRanker model colbert-ir/colbertv2.0


In [26]:
def rank_results(encoder, prompt, docs, doc_ids, top_k=10):
    ranker = ENCODER_MODELS[encoder]
    results = ranker.rank(query=prompt, docs=docs, doc_ids=doc_ids).top_k(top_k)
    # print(f"Got {len(results.results)} results")
    return results

def show(results):
    for r in results:
        print(f"{r.doc_id}  {BeautifulSoup(r.text, 'html.parser').text}")
        # print(BeautifulSoup(r.text, "html.parser").text)

In [19]:
prompt = "What is Artemis?"

In [20]:
num_chunks = 20
small_chunks = chunks[0:10]
chunk_text = [c.clean_text for c in small_chunks][:num_chunks]
chunk_ids = [c.index for c in small_chunks][:num_chunks]

In [27]:
%%time
res_ce = rank_results("cross-encoder", prompt, chunk_text, chunk_ids)

CPU times: total: 16.3 s
Wall time: 4.44 s


In [28]:
%%time
res_col = rank_results("colbert", prompt, chunk_text, chunk_ids)

CPU times: total: 16.1 s
Wall time: 4.07 s


In [29]:
show(res_ce)
show(res_col)

19444  Overall,the CFD results demonstrate that scale-resolving simulation methodologies like the one presentedcan help characterize full-scale launch abort vehicle near-eld noise and can complement windtunnel, ground and ight tests by providing trends and reducing uncertainty for scenarios thatcannot or are not planned to be tested.Keywords: Computational Fluid Dynamics, Aero-acoustics.1 IntroductionThe National Aeronautics and Space Administrations (NASA) Artemis missions plan to bring humans tothe Moon  and eventually Mars  and return them to Earth safely onboard the Orion spacecraft.  Orionwill be propelled into space by the powerful Space Launch System, shown in Figure 1 with the Orion crewmodule and its launch abort system  the focus of this work  outlined in red.  The launch abort system(LAS), detailed in Figure 2 is attached to the top of the Orion crew module during launch and is jettisonedonce the spacecraft safely reaches the edge of space. 
19445  Orionwill be propelled int

## Build QueryReferences

In [ ]:
query_result = QueryResult.find_by_id("0O5cJA8xJRXcSobdsZcJ")
query_ref_ids = query_result.query_refs

In [ ]:
query_ref_ids

In [ ]:
query_ref_text = []
query_refs = []

for query_ref_id in query_ref_ids:
    query_ref = QueryReference.find_by_id(query_ref_id)
    query_doc_chunk = QueryDocumentChunk.find_by_id(query_ref.chunk_id)
    print(query_ref.id, query_ref_id, query_ref.chunk_id, query_doc_chunk.id)
    query_ref_text.append(query_doc_chunk.clean_text)
    query_refs.append(query_ref)


## Try reranking

In [ ]:
RERANK_MODEL_NAME = "cross-encoder"
reranker = Reranker(RERANK_MODEL_NAME, verbose=0)

In [ ]:
ranked_results = reranker.rank(query=prompt_one, docs=query_ref_text, doc_ids=query_ref_ids)

In [ ]:
ranked_query_ref_ids = [ r.doc_id for r in ranked_results.results ]
ranked_query_ref_ids

In [ ]:
sort_order_dict = {x: i for i, x in enumerate(ranked_query_ref_ids)}
sort_order_dict

In [ ]:
sort_order_list = [(qr, sort_order_dict[qr.id]) for qr in query_refs]
sort_order_list
sort_order_list.sort(key=lambda x: x[1])
sort_order_list

ranked_query_refs = [qr for qr, i in sort_order_list]

print([qr.id for qr in ranked_query_refs])

In [ ]:
def rank_references2(prompt: str,
                     query_references: List[QueryReference]) -> List[QueryReference]:

    query_ref_text = []
    query_ref_ids = []

    for query_ref in query_references:
        # query_ref = QueryReference.find_by_id(query_ref_id)
        query_doc_chunk = QueryDocumentChunk.find_by_id(query_ref.chunk_id)
        # print(query_ref.id, query_ref_id, query_ref.chunk_id, query_doc_chunk.id)
        query_ref_text.append(query_doc_chunk.clean_text)
        query_ref_ids.append(query_ref.id)

    ranked_results = reranker.rank(query=prompt, docs=query_ref_text, doc_ids=query_ref_ids)
    # putting the references list in order by rank
    ranked_query_ref_ids = [ r.doc_id for r in ranked_results.results ]
    sort_dict = {x: i for i, x in enumerate(ranked_query_ref_ids)}
    sort_list = [(qr, sort_dict[qr.id]) for qr in query_refs]
    sort_list.sort(key=lambda x: x[1])

    # just return the QueryReferences
    ranked_query_refs = [qr for qr, i in sort_order_list]

    return ranked_query_refs

In [ ]:
query_result = QueryResult.find_by_id("0O5cJA8xJRXcSobdsZcJ")
query_ref_ids = query_result.query_refs

query_refs = []
for query_ref_id in query_ref_ids:
    print(query_ref_id)
    query_refs.append(QueryReference.find_by_id(query_ref_id))

ranked_query_refs = rank_references2(prompt_one, query_refs)

In [ ]:
for qr in ranked_query_refs:
    print(qr.id)

## Test reranking function

In [ ]:
from services.query.query_service import rerank_references

In [ ]:
ranked_query_refs = rerank_references(prompt_one, query_refs)

In [ ]:
for r in ranked_query_refs:
    print(r.id, r.chunk_id)